In [59]:
import pandas as pd
import numpy as np
import random
import torch
import transformers
import torch.nn as nn
from torch.utils.data import Dataset
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor, ToPILImage
from torchvision.io import read_image
from transformers import TrainingArguments, Trainer, AutoModel, AutoModelForImageClassification, AutoImageProcessor
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.metrics import f1_score
import os
from PIL import Image
import matplotlib.pyplot as plt

In [26]:
model_checkpoint = "google/vit-base-patch16-224-in21k"
path_to_images = "/content/drive/MyDrive/projects/ai_images_detector/images/"

In [4]:
model = AutoModelForImageClassification.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
model

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=7

In [6]:
data_path = 'train.csv'

df = pd.read_csv(data_path)
df

,id,target
0,gL5fWrgNPU.jpg,0
1,KKWXJDN8uM.png,1
2,Wb2I0CXlAY.jpg,0
3,G7PLhKpaf7.jpeg,1
4,DEUkeefz6Z.jpg,0
...,...,...
1007,fqoVycdxLV,1
1008,TN2ZOiNI5e.jpg,0
1009,2WMWt6CAWQ.jpg,0
1010,fQCY5n2vcF.jpg,0


In [41]:
df.iloc[3][0]

'G7PLhKpaf7.jpeg'

In [9]:
image_processor = AutoImageProcessor.from_pretrained(model_checkpoint)

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

In [29]:
'''def images_to_pil(path, mode = "Ai+Real"):
  pixel_values = []
  labels = []
  if mode == "Ai+Real":
    for name in df["id"]:
      if "." in name:
        pixel_values.append(Image.open(path + name))
        labels.append(df[df['id']==name].iloc[0].target)
      else:
        for image in os.listdir(path):
          if image.split('.')[0] == name:
            pixel_values.append(Image.open(path + image))
            labels.append(df[df['id']==name].iloc[0].target)
            break
  return pixel_values, labels

x, y = images_to_pil(path = path_to_images)'''

In [56]:
normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
size = (
    image_processor.size["shortest_edge"]
    if "shortest_edge" in image_processor.size
    else (image_processor.size["height"], image_processor.size["width"])
)
transforms = Compose([RandomResizedCrop(size), ToTensor()])


In [57]:
class ImageDataset(Dataset):
  def __init__(self, img_dir, transform, annotations_file='train.csv', mode = "Ai+Real"):
    self.mode = mode
    if mode == "Ai+Real":
      self.img_info = pd.read_csv(annotations_file)
    self.img_dir = img_dir
    self.transform = transform

  def __len__(self):
    return len(self.img_info)

  def __getitem__(self, idx):
    name = self.img_info.iloc[idx, 0]
    if "." in name:
      img_path = os.path.join(self.img_dir, name)
    else:
      for imgname in os.listdir(self.img_dir):
        if imgname.split('.')[0] == name:
          img_path = os.path.join(self.img_dir, imgname)
          break
    pil_image = Image.open(img_path)
    label = self.img_info.iloc[idx, 1]
    image = self.transform(pil_image)
    return image, label

In [61]:
data = ImageDataset(path_to_images, transforms, annotations_file='train.csv')
tr = ToPILImage()
img = tr(data[0][0])
type(img)

PIL.Image.Image